## Setting up the environment

In [2]:
# authenticate EE
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=LFmjZpm-MY1BjREj3Z7AGahv8pZ6QHi0xPI7EBk8HAQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5c57jpTusFDEP3NiHWU8w1YKcj2Vpv4HmYvOCtWdC-pc6PKR87Yx8

Successfully saved authorization token.


In [3]:
# authenticate and mount Google Drive folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [106]:
# Import the Earth Engine API and initialize it.
import ee
from ee import batch
ee.Initialize()

## Functions

In [107]:
''' Check that Earth Engine is available '''
try:     # try to load the Landsat 8 Tier 1 EE image collection
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
except:  # if the loading image collection fails, then initialize EE in the code
    ee.Initialize()

'''
Functions to mask clouds based on pixel_qa band of Landsat 8 SR data borrowed from:
https://gis.stackexchange.com/questions/274048/apply-cloud-mask-to-landsat-imagery-in-google-earth-engine-python-api

'''
# function to get QA data from image
def getQABits(image, start, end, mascara):
    # compute the bits we need to extract
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # return single band image of the extracted QA bits, giving the band a new name
    return image.select([0],[mascara]).bitwiseAnd(pattern).rightShift(start)

def maskQuality(image):
    # select the QA band
    QA = image.select('pixel_qa')
    # get the internal_cloud_algorithm_flag bit
    sombra = getQABits(QA,3,3,'cloud_shadow')
    nubes = getQABits(QA,5,5,'cloud')
    # detect cirrus clouds using bit 9
    cirrus_detected = getQABits(QA,9,9,'cirrus_detected')
    # return an image masking out cloudy areas
    return image.updateMask(sombra.eq(0)).updateMask(nubes.eq(0).updateMask(cirrus_detected.eq(0)))


## Spatial and Temporal Filters and Other User Options

In [108]:
# create area of interest polygon (several options presented below)
## Example 1: Simple bounding box for the entire Gulf of Mexico
# aoipoly = ee.Geometry.BBox(-98, 26,-82,31)

## Example 2: Bounding polygon for area around Pensacola, FL, USA
aoipoly = ee.Geometry.Polygon([[[-88.412770, 31.098514], 
                                [-87.015281, 31.098514], 
                                [-87.015281, 30.156911], 
                                [-88.412770, 30.156911], 
                                [-88.412770, 31.098514]]])

## Example 3: Bounding polygon for entire Gulf of Mexico (USA) coast
# aoipoly = ee.Geometry.Polygon([[[-98.347274, 26.322097], 
#                                 [-98.238364, 28.187734], 
#                                 [-95.430534, 32.017493], 
#                                 [-84.011970, 32.564172], 
#                                 [-82.679402, 31.166281], 
#                                 [-80.410998, 24.154884], 
#                                 [-86.730146, 28.166929], 
#                                 [-94.552293, 27.623251], 
#                                 [-96.338964, 25.553449], 
#                                 [-98.347274, 26.322097]]])

# start and end dates for filtering imagery
START_DATE = '2016-01-01'
END_DATE = '2021-04-30'

# optional flag to enable data downloading
download_data = True

## Sample Data (download optional)

### Landsat 8 Tier 1 imagery

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR

In [109]:
# specify the image collection (Landsat 8) (with spatial and temporal filters)
l8t1_tmp = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
                .filterDate(START_DATE, END_DATE)\
                .filterBounds(aoipoly)\
                .map(maskQuality)
# combine multiple images by taking the median of all available images, limit data
# to area of interest, and force 8-bit conversion
l8t1 = l8t1_tmp.median().clip(aoipoly)

if download_data:
  # create download container to bulk/batch export the download imagery to Drive
  l8t1_download = ee.batch.Export.image.toDrive(
      image = l8t1,              # specify data to be exported
      description = 'Landsat8_Tier1_imagery', # give the image a description
      folder = 'delta_dat',         # linked Google Drive folder
      region = aoipoly,             # limit output to specified geometry
      scale = 30,                   # spatial resolution of data
      fileNamePrefix = 'landsat8',  # filename for output(s)
      crs = 'EPSG:4326',            # specify projection
      maxPixels = 1e13,             # increase the number of pixel limit
      fileFormat = 'GeoTIFF',       # specify output file format
  )

  # start data download
  l8t1_download.start()

In [110]:
# check the status of the data download (if download_data is set to True)
l8t1_download.status()

{'creation_timestamp_ms': 1621628042825,
 'description': 'Landsat8_Tier1_imagery',
 'id': 'FGHAVNF7RPMAABA2K6M5XP44',
 'name': 'projects/earthengine-legacy/operations/FGHAVNF7RPMAABA2K6M5XP44',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621628042825}

### US National Landcover

https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2016_REL

In [111]:
# EE location of NLCD land cover (with spatial filter)
nlcd = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')\
              .filterBounds(aoipoly)

# user-specified year of interest
yoi = '2016'

# filter data to 2016 and landcover band
nlcd2016 = nlcd.filter(ee.Filter.eq('system:index',yoi)).first()
landcover = nlcd2016.select('landcover').clip(aoipoly)

if download_data:
  # create download container to bulk/batch export the download imagery to Drive
  nlcd_download = ee.batch.Export.image.toDrive(
      image = landcover,
      description = 'Landcover',
      folder = 'delta_dat',
      region = aoipoly,
      scale = 30,
      fileNamePrefix = 'lulc',
      crs = 'EPSG:4326',
      maxPixels = 1e13,
      fileFormat = 'GeoTIFF',
  )

  # start data download
  nlcd_download.start()

In [112]:
# check the status of the data download (if download_data is set to True)
nlcd_download.status()

{'creation_timestamp_ms': 1621628047564,
 'description': 'Landcover',
 'id': 'X4WDEKACASGYIRQPQZJ4B2MS',
 'name': 'projects/earthengine-legacy/operations/X4WDEKACASGYIRQPQZJ4B2MS',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621628047564}

### DEM

Multi-Error-Removed Improved-Terrain (MERIT) DEM

https://developers.google.com/earth-engine/datasets/catalog/MERIT_DEM_v1_0_3

In [113]:
# EE location of MERIT DEM (with spatial filter)
mdem = ee.Image("MERIT/DEM/v1_0_3")\
              .clip(aoipoly)

if download_data:
  # # create download container to bulk/batch export the download imagery to Drive
  mdem_download = ee.batch.Export.image.toDrive(
      image = mdem,
      description = 'MERIT',
      folder = 'delta_dat',
      region = aoipoly,
      scale = 30,
      fileNamePrefix = 'merit_dem',
      crs = 'EPSG:4326',
      maxPixels = 1e13,
      fileFormat = 'GeoTIFF',
  )

  # start data download
  mdem_download.start()

In [114]:
# check the status of the data download (if download_data is set to True)
mdem_download.status()

{'creation_timestamp_ms': 1621628051074,
 'description': 'MERIT',
 'id': 'CDNWRFJ3ILKD6U6YGYD2VLKC',
 'name': 'projects/earthengine-legacy/operations/CDNWRFJ3ILKD6U6YGYD2VLKC',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621628051074}

### Topographic Diversity Index

Global SRTM Topographic Diversity

https://developers.google.com/earth-engine/datasets/catalog/CSP_ERGo_1_0_Global_SRTM_topoDiversity

In [115]:
# EE location of toppographic diversity index layer (with spatial filter)
tdi = ee.Image("CSP/ERGo/1_0/Global/SRTM_topoDiversity")\
              .select('constant')\
              .clip(aoipoly)

if download_data:
  # # create download container to bulk/batch export the download imagery to Drive
  tdi_download = ee.batch.Export.image.toDrive(
      image = lith,
      description = 'SRTM Topographic Diversity Index',
      folder = 'delta_dat',
      region = aoipoly,
      scale = 30,
      fileNamePrefix = 'tdi',
      crs = 'EPSG:4326',
      maxPixels = 1e13,
      fileFormat = 'GeoTIFF',
  )

  # start data download
  tdi_download.start()

In [116]:
# check the status of the data download (if download_data is set to True)
tdi_download.status()

{'creation_timestamp_ms': 1621628054069,
 'description': 'SRTM Topographic Diversity Index',
 'id': 'FGTWQCTGGOLQ2AXORNF3GKK4',
 'name': 'projects/earthengine-legacy/operations/FGTWQCTGGOLQ2AXORNF3GKK4',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621628054069}

### Topographic Position Index

Global SRTM mTPI (Multi-Scale Topographic Position Index)

https://developers.google.com/earth-engine/datasets/catalog/CSP_ERGo_1_0_Global_SRTM_mTPI

In [117]:
# EE location of topographic position index (with spatial filter)
mtpi =  ee.Image("CSP/ERGo/1_0/Global/SRTM_mTPI")\
              .select('elevation')\
              .clip(aoipoly)

if download_data:
  # # create download container to bulk/batch export the download imagery to Drive
  mtpi_download = ee.batch.Export.image.toDrive(
      image = lith,
      description = 'Multi-scale Topographic Position Index',
      folder = 'delta_dat',
      region = aoipoly,
      scale = 30,
      fileNamePrefix = 'mtpi',
      crs = 'EPSG:4326',
      maxPixels = 1e13,
      fileFormat = 'GeoTIFF',
  )

  # start data download
  mtpi_download.start()

In [118]:
# check the status of the data download (if download_data is set to True)
mtpi_download.status()

{'creation_timestamp_ms': 1621628057147,
 'description': 'Multi-scale Topographic Position Index',
 'id': 'DJ7SBLUZWDGKPAYJEKDJCCCA',
 'name': 'projects/earthengine-legacy/operations/DJ7SBLUZWDGKPAYJEKDJCCCA',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621628057147}

### Soil Taxonomy Great Groups

OpenLandMap USDA soil taxonomy great groups

https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_SOL_SOL_GRTGROUP_USDA-SOILTAX_C_v01

In [119]:
# EE location of soil taxonomy (with spatial filter)
soiltax = ee.Image("OpenLandMap/SOL/SOL_GRTGROUP_USDA-SOILTAX_C/v01")\
              .select('grtgroup')\
              .clip(aoipoly)

if download_data:
  # # create download container to bulk/batch export the download imagery to Drive
  soiltax_download = ee.batch.Export.image.toDrive(
      image = lith,
      description = 'USDA Soil Taxonomy Great Groups',
      folder = 'delta_dat',
      region = aoipoly,
      scale = 30,
      fileNamePrefix = 'soiltax',
      crs = 'EPSG:4326',
      maxPixels = 1e13,
      fileFormat = 'GeoTIFF',
  )

  # start data download
  soiltax_download.start()

In [120]:
# check the status of the data download (if download_data is set to True)
soiltax_download.status()

{'creation_timestamp_ms': 1621628060692,
 'description': 'USDA Soil Taxonomy Great Groups',
 'id': 'G4XLA4X555CNK5LEUKDMAMXK',
 'name': 'projects/earthengine-legacy/operations/G4XLA4X555CNK5LEUKDMAMXK',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621628060692}

### Soil Organic Carbon Content

OpenLandMap Soil organic carbon content

https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_SOL_SOL_ORGANIC-CARBON_USDA-6A1C_M_v02

In [121]:
# EE location of soil Carbon (with spatial filter)
soilc = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02")\
              .select(['b0','b10','b30','b60','b100','b200'])\
              .clip(aoipoly)

if download_data:
  # # create download container to bulk/batch export the download imagery to Drive
  soilc_download = ee.batch.Export.image.toDrive(
      image = lith,
      description = 'USDA Soil Taxonomy Great Groups',
      folder = 'delta_dat',
      region = aoipoly,
      scale = 30,
      fileNamePrefix = 'soilc',
      crs = 'EPSG:4326',
      maxPixels = 1e13,
      fileFormat = 'GeoTIFF',
  )

  # start data download
  soilc_download.start()

In [122]:
# check the status of the data download (if download_data is set to True)
soilc_download.status()

{'creation_timestamp_ms': 1621628064187,
 'description': 'USDA Soil Taxonomy Great Groups',
 'id': 'UFZZVJ2FCWK2NXXAFN6AKXGO',
 'name': 'projects/earthengine-legacy/operations/UFZZVJ2FCWK2NXXAFN6AKXGO',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621628064187}

### Soil Water Content

OpenLandMap Soil water content at 33kPa (field capacity)

https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_SOL_SOL_WATERCONTENT-33KPA_USDA-4B1C_M_v01

In [123]:
# EE location of soil Carbon (with spatial filter)
soilwater = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01")\
              .select(['b0','b10','b30','b60','b100','b200'])\
              .clip(aoipoly)

if download_data:
  # # create download container to bulk/batch export the download imagery to Drive
  soilwater_download = ee.batch.Export.image.toDrive(
      image = lith,
      description = 'USDA Soil Water Content',
      folder = 'delta_dat',
      region = aoipoly,
      scale = 30,
      fileNamePrefix = 'soilwater',
      crs = 'EPSG:4326',
      maxPixels = 1e13,
      fileFormat = 'GeoTIFF',
  )

  # start data download
  soilwater_download.start()

In [124]:
# check the status of the data download (if download_data is set to True)
soilwater_download.status()

{'creation_timestamp_ms': 1621628068377,
 'description': 'USDA Soil Water Content',
 'id': 'U2WE7FBJK25LNCSGDY5BDVM5',
 'name': 'projects/earthengine-legacy/operations/U2WE7FBJK25LNCSGDY5BDVM5',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1621628068377}

## Cloud Analysis

In [125]:
'''
Available prepped data and bands:
landsat 8 tier 1                  -->   l8t1
    B1    ultra B         0.435-0.451 μm
    B2    B               0.452-0.512 μm
    B3    G               0.533-0.590 μm
    B4    R               0.636-0.673 μm
    B5    NIR             0.851-0.879 μm
    B6    SWIR 1          1.566-1.651 μm
    B7    SWIR 2          2.107-2.294 μm
    B10*  Temperature 1   10.60-11.19 μm
    B11*  Temperature 2   11.50-12.51 μm
    sr_aerosol            Aerosol attributes
      *bands were originally collected at 100m resolution and were resampled
       to 30m using cubic convolution (pre-processed before accessing using EE)
landcover                         -->   landcover
    landcover             Landcover classification scheme from
    https://www.mrlc.gov/data/legends/national-land-cover-database-2016-nlcd2016-legend
MERIT dem                         -->   mdem
    dem                   3 arc seconds
    (elevation in meters, referenced to EGM96 geoid model)
STRM topographic diversity index  -->   tdi
    tdi                   SRTM-derived topographic diversity index 
STRM topographic position index   -->   mtpi
    tpi                   SRTM-derived topographic position index
USDA soil taxonomy great group    -->   soiltax
    grtgroup              USDA soil taxonomy great group name
USDA soil organic carbon content  -->   soilc
    b0      g/kg          Soil organic carbon content at 0 cm depth
    b10     g/kg          Soil organic carbon content at 10 cm depth
    b30     g/kg          Soil organic carbon content at 30 cm depth
    b60     g/kg          Soil organic carbon content at 60 cm depth
    b100    g/kg          Soil organic carbon content at 100 cm depth
    b200    g/kg          Soil organic carbon content at 200 cm depth
USDA soil water content           -->   soilwater
    b0      %           Soil water content at 33kPa (field capacity) at 0 cm depth
    b10     %           Soil water content at 33kPa (field capacity) at 10 cm depth
    b30     %           Soil water content at 33kPa (field capacity) at 30 cm depth
    b60     %           Soil water content at 33kPa (field capacity) at 60 cm depth
    b100    %           Soil water content at 33kPa (field capacity) at 100 cm depth
    b200    %           Soil water content at 33kPa (field capacity) at 200 cm depth
'''

# create raster with landsat 8, landcover, dem, and tdi bands
dat = l8t1.addBands(landcover)\
              .addBands(mdem)\
              .addBands(tdi)\
              .addBands(mtpi)\
              .addBands(soiltax)\
              .addBands(soilc)\
              .addBands(soilwater)

In [126]:
# print names of all bands in new composite
print(dat.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa', 'landcover', 'dem', 'constant', 'elevation', 'grtgroup', 'b0', 'b10', 'b30', 'b60', 'b100', 'b200', 'b0_1', 'b10_1', 'b30_1', 'b60_1', 'b100_1', 'b200_1']
